# 🎉 Spontaneous State Simulation

---
## 🔍 Notebook objectives

This notebook contains simulation of spontaneous state in the ring attractor network which contains 360 excitatory RS neurons and 90 inhibitory FS interneurons.

# 🎒 Setup

## ⬇️ Imports

In [ ]:
from utils.main_simulation import network_simulation_run
from utils.poisson_spike_train_generation import generate_spike_train
from utils.plots import *

import numpy as np
import yaml #reading env consts

## 🛠️ Simulation Utils

In [ ]:
#RS neuron parameters
with open('utils/RS.yaml', 'r', encoding="utf-8") as f:
    params_RS = yaml.load(f, Loader=yaml.FullLoader)

#FS neuron parameters
with open('utils/FS.yaml', 'r', encoding="utf-8") as f:
    params_FS = yaml.load(f, Loader=yaml.FullLoader)

#receptor kinetics parameters
with open('utils/receptor_kinetics.yaml', 'r', encoding="utf-8") as f:
    params_receptor_kinetics = yaml.load(f, Loader=yaml.FullLoader)

#synaptic weights parameters
with open('utils/synaptic_weights.yaml', 'r', encoding="utf-8") as f:
    params_synaptic_weights = yaml.load(f, Loader=yaml.FullLoader)

#external input parameters
with open('utils/external_input.yaml', 'r', encoding="utf-8") as f:
    params_external_input = yaml.load(f, Loader=yaml.FullLoader)

#stimuli input parameters
with open('utils/stimuli_input.yaml', 'r', encoding="utf-8") as f:
    params_stimuli_input = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
#simulation time; always 5 seconds, for all experiments
seconds = 5

t_min = 0
t_max = int(seconds*1000) #in ms -> 1(s) of simulation
delta_T = 0.1 #0.1 ms is integration step
sim_steps = int(seconds*1000/delta_T)

T = np.linspace(t_min, t_max, sim_steps)

# 🧪 Experiment

### 🏋️ Synaptic Weights

Definition of synaptic weights.

In [ ]:
#360 RS and 90 FS
Ne = 360
Ni = 90

weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j

# e -> e
for i in range(Ne):
    for j in range(Ne):
        angle_i = i / Ne * 360
        angle_j = j / Ne * 360
        weights[i, j] = params_synaptic_weights['j_ee'] * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/params_synaptic_weights['sigma']**2)

# e -> i
for i in range(Ne):
    for j in range(Ni):
        angle_i = i / Ne * 360
        angle_j = j / Ni * 360
        weights[i, Ne + j] = params_synaptic_weights['j_ei']

# i -> e
for i in range(Ni):
    for j in range(Ne):
        angle_i = i / Ni * 360
        angle_j = j / Ne * 360
        weights[Ne + i, j] = params_synaptic_weights['j_ie']
        
# i -> i
for i in range(Ni):
    for j in range(Ni):
        angle_i = i / Ni * 360
        angle_j = j / Ni * 360
        weights[Ne + i, Ne + j] = params_synaptic_weights['j_ii']

### 🚝 External Cortical Input & 🎯 Stimulus Representation

Observe that we don't present any external stimuli in the case of spontaneous state activity, only background cortical input.

In [ ]:
v = np.array(np.zeros((len(T), Ne + Ni)))
v[:, :Ne] = params_external_input['v_e']
v[:, Ne:] = params_external_input['v_i']

### 🥳 Serotonin Weights

All of the serotonin weights are defined to be default.

In [ ]:
serotonin_weights = np.ones(Ne + Ni)

### 🔩 Settings

In [ ]:
start_state = np.column_stack((np.append(params_RS["v"]*np.ones(Ne), params_FS["v"]*np.ones(Ni)), np.append(params_RS["u"]*np.ones(Ne), params_FS["u"]*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {"a": np.append(params_RS["a"]*np.ones(Ne), params_FS["a"]*np.ones(Ni)), 
          "b": np.append(params_RS["b"]*np.ones(Ne), params_FS["b"]*np.ones(Ni)), 
          "c": np.append(params_RS["c"]*np.ones(Ne), params_FS["c"]*np.ones(Ni)), 
          "d": np.append(params_RS["d"]*np.ones(Ne), params_FS["d"]*np.ones(Ni)), 
          "C": np.append(params_RS["C"]*np.ones(Ne), params_FS["C"]*np.ones(Ni)), 
          "k": np.append(params_RS["k"]*np.ones(Ne), params_FS["k"]*np.ones(Ni)),
          "v_peak": np.append(params_RS["v_peak"]*np.ones(Ne), params_FS["v_peak"]*np.ones(Ni)), 
          "v_r": np.append(params_RS["v_r"]*np.ones(Ne), params_FS["v_r"]*np.ones(Ni)), 
          "v_t": np.append(params_RS["v_t"]*np.ones(Ne), params_FS["v_t"]*np.ones(Ni)),
          "tau_ampa": params_receptor_kinetics["tau_ampa"]*np.ones((Ne + Ni)),
          "tau_nmda": params_receptor_kinetics["tau_nmda"]*np.ones((Ne + Ni)),
          "tau_gabaa": params_receptor_kinetics["tau_gabaa"]*np.ones((Ne + Ni)),
          "tau_gabab": params_receptor_kinetics["tau_gabab"]*np.ones((Ne + Ni)),
          "g_ampa": params_receptor_kinetics["g_ampa"]*np.ones((Ne + Ni)),
          "g_nmda": params_receptor_kinetics["g_nmda"]*np.ones((Ne + Ni)),
          "g_gabaa": params_receptor_kinetics["g_gabaa"]*np.ones((Ne + Ni)),
          "g_gabab": params_receptor_kinetics["g_gabab"]*np.ones((Ne + Ni)),
          "g_e_external": params_external_input["g_e_external"]*np.ones((Ne + Ni)),
          "g_i_external": params_external_input["g_i_external"]*np.ones((Ne + Ni))
          }

### 🏃 Run

It might take about 15 minutes to complete (depending on the hardware you are using).

In [ ]:
sim_num = 30 #30 - default

In [ ]:
np.random.seed(42) #for reproducibility

exc_firing_rates = []
inh_firing_rates = []

for _ in range(sim_num):
    spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)
    _, firings, _, _, _, _, _ = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
    observed_firing_rates = np.sum(firings, axis = 0) / seconds
    exc_firing_rates += list(observed_firing_rates[:360])
    inh_firing_rates += list(observed_firing_rates[360:])

### 👁️ Visualizations

To reproduce Figure 5.1 from thesis.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

trace1 = go.Box(y=exc_firing_rates, name='Excitatory firing rates')
trace2 = go.Box(y=inh_firing_rates, name='Inhibitory firing rates')
data = [trace1, trace2]

layout = go.Layout(
    title='Spontaneous State Activity',
    yaxis=dict(title='Firing Rate (in Hz)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600)
fig.show()

In [ ]:
print(f"Mean value for excitatory firing rate is: {np.mean(exc_firing_rates):.02f} (Hz).")
print(f"Mean value for inhibitory firing rate is: {np.mean(inh_firing_rates):.02f} (Hz).")

### 👁️ Visualizations for Particular Run

In [ ]:
np.random.seed(42) #for reproducibility
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)
_, firings, _, _, _, _, _ = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)

To reproduce Figure 5.2 from thesis.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

fig = exc_inh_firing_rates(firings[:, :360], firings[:, 360:], [], [], delta_T)
fig.update_layout(height = 600)
fig.show()